# jsonpath_ng lib

A biblioteca **`jsonpath_ng`** em Python é uma implementação robusta e poderosa da linguagem **JSONPath**. Como vimos antes, JSONPath é uma linguagem de consulta para JSON, similar ao XPath para XML, permitindo que você selecione e extraia partes específicas de um documento JSON de forma declarativa.

`jsonpath_ng` se destaca por ser uma implementação flexível e rica em recursos, que se esforça para ser compatível com a especificação JSONPath, mas com algumas nuances e extensões que a tornam muito útil em ambientes Python.

## Como `jsonpath_ng` Funciona?

O processo geral de uso de `jsonpath_ng` envolve duas etapas principais:

1.  **Parsing da Expressão JSONPath:** Primeiro, você "compila" sua string de expressão JSONPath em um objeto de expressão `jsonpath_ng`. Isso permite que a biblioteca entenda a estrutura da sua consulta.
2.  **Busca no Documento JSON:** Em seguida, você usa o objeto de expressão compilada para buscar (ou "encontrar") os valores correspondentes dentro de um documento JSON (geralmente um dicionário ou lista Python).

A biblioteca retorna os resultados como uma lista de objetos `Match`, onde cada `Match` contém o valor encontrado e informações sobre seu caminho no documento original.


In [1]:
%pip install jsonpath-ng

Note: you may need to restart the kernel to use updated packages.


## Classes e Métodos Principais

Vamos mergulhar nas principais classes, métodos e propriedades com exemplos práticos.

Primeiro, preparemos o documento JSON de exemplo que usamos anteriormente:


In [2]:
import json
import time
from jsonpath_ng import jsonpath, parse
from jsonpath_ng.ext import parse as ext_parse
from jsonpath_ng.jsonpath import Index, Fields, Child, Where, Union  # Descendant
from jsonpath_ng.lexer import JsonPathLexer
from data import json_data  # data.py

In [3]:
def run_jsonpath_query(query: str):
    """
    Executes a JSONPath query using jsonpath-ng and prints the results.
    """
    print(f"\n--- JSONPath Query: '{query}' ---")
    try:
        jsonpath_expr = parse(query)
        matches = jsonpath_expr.find(json_data)
        if matches:
            print(f"  Resultados ({len(matches)}):")
            for i, match in enumerate(matches):
                # O objeto 'match' tem propriedades 'value' e 'path'
                print(
                    f"    Match {i+1}: Valor = {match.value}, Caminho = {match.path}")
        else:
            print("  Nenhum resultado encontrado.")
    except Exception as e:
        print(f"  Erro ao executar query: {e}")


def separador(titulo):
    """Função auxiliar para separar seções"""
    print(f"\n{'='*80}")
    print(f" {titulo}")
    print(f"{'='*80}")


def exemplo(descricao, codigo, resultado):
    """Função auxiliar para mostrar exemplos"""
    print(f"\n📌 {descricao}")
    print(f"Código: {codigo}")
    print(f"Resultado: {resultado}")
    
def tr_dict(obj):
    out=''
    for i in obj:
        out+=f'\n  - {i}: {obj[i]}'
    return out
    

### 1. Funções de Parsing e Busca

  - **`jsonpath_ng.parse(jsonpath_string)`**:

      - **O que faz:** Recebe uma string que representa uma expressão JSONPath e a compila em um objeto de expressão. Este é o ponto de partida para qualquer consulta.
      - **Retorna:** Uma instância de `jsonpath_ng.JsonPath` (ou uma de suas subclasses, como `Child`, `Slice`, `Filter`, etc.).

  - **`jsonpath_expr.find(data)`**:

      - **O que faz:** É um método do objeto de expressão JSONPath compilado. Ele executa a consulta JSONPath no `data` (seu documento JSON, que deve ser um dicionário ou lista Python) e retorna todos os `Match`es encontrados.
      - **`data`**: O objeto Python (geralmente um `dict` ou `list`) no qual a pesquisa será realizada.
      - **Retorna:** Uma lista de objetos `jsonpath_ng.jsonpath.Match`.


In [4]:
# Exemplo 1: Selecionando um valor simples
run_jsonpath_query('$.livraria.titulo')

# Exemplo 2: Selecionando todos os títulos de livros
run_jsonpath_query('$.livraria.livros[*].titulo')

# Exemplo 3: Selecionando o primeiro autor do segundo livro
run_jsonpath_query('$.livraria.livros[1].autores[0]')


--- JSONPath Query: '$.livraria.titulo' ---
  Resultados (1):
    Match 1: Valor = Catálogo de Livros Técnicos, Caminho = titulo

--- JSONPath Query: '$.livraria.livros[*].titulo' ---
  Resultados (4):
    Match 1: Valor = Aprendendo JSONPath, Caminho = titulo
    Match 2: Valor = Dominando JMESPath, Caminho = titulo
    Match 3: Valor = Introdução ao Web Scraping, Caminho = titulo
    Match 4: Valor = Python para Iniciantes, Caminho = titulo

--- JSONPath Query: '$.livraria.livros[1].autores[0]' ---
  Resultados (1):
    Match 1: Valor = Charlie, Caminho = [0]


### 2. O Objeto `Match`

Quando `find()` retorna resultados, eles vêm na forma de uma lista de objetos `Match`. Um objeto `Match` tem duas propriedades principais:

  - **`match.value`**: O valor real encontrado no documento JSON.
  - **`match.path`**: O objeto de caminho (uma instância de `jsonpath_ng.jsonpath.JsonPath`) que levou a este valor. Pode ser útil para entender de onde o valor veio. Para obter o caminho como string, você pode usar `str(match.path)`.

**Exemplos:** (Os exemplos acima já demonstram o uso de `match.value` e `match.path`)


### 3. Sintaxe e Recursos JSONPath Comuns em `jsonpath_ng`

`jsonpath_ng` suporta a maioria dos elementos da sintaxe JSONPath.

  - **`$`**: O elemento raiz do objeto JSON.
  - **`.`**: Operador de ponto, usado para acessar membros de objeto. Ex: `$.livraria.titulo`
  - **`[]`**: Operador de subscrição para arrays ou para chaves de objeto com caracteres especiais.
      - **`[index]`**: Acessa um elemento de array pelo índice. Ex: `$.livraria.livros[0]`
      - **`[start:end:step]`**: Fatiamento de array (slicing). Ex: `$.livraria.livros[0:2]`
      - **`['key-with-hyphen']`**: Acessa uma chave de objeto com caracteres especiais. Ex: `$.configuracoes['chave-com-traco']`
      - **`[index1, index2]`**: Seleção de múltiplos índices de array. Ex: `$.livraria.livros[0,2].titulo`
  - **`*`**: Wildcard para todos os membros de objeto ou todos os elementos de array. Ex: `$.livraria.livros[*].titulo`
  - **`..`**: Operador de descendência recursiva. Busca o nome em qualquer nível abaixo do atual. (Cuidado com o desempenho em documentos grandes). Ex: `$.livraria..titulo` (encontraria todos os campos `titulo` em qualquer profundidade).
  - **`?()`**: Expressão de filtro. Permite filtrar elementos de um array ou membros de objeto baseados em uma condição.
      - **`@`**: Representa o elemento atual no filtro.
      - **Operadores de comparação:** `==`, `!=`, `<`, `<=`, `>`, `>=`.
      - **Operadores lógicos:** `&&` (AND), `||` (OR).
      - **Expressões regulares:** Alguns parsers suportam `~= /regex/`. `jsonpath-ng` usa `@.field =~ "regex"`.


In [5]:
# Selecionando chaves com caracteres especiais
run_jsonpath_query('$.configuracoes["chave-com-traco"]')

# Fatiamento de array (slicing): os dois primeiros livros
run_jsonpath_query('$.livraria.livros[0:2].titulo')

# Seleção de múltiplos índices: primeiro e terceiro livro
run_jsonpath_query('$.livraria.livros[0,2].titulo')

# Descendência recursiva: todos os títulos em qualquer nível
# Este pode ser pesado para documentos muito grandes!
run_jsonpath_query('$.livraria..titulo')

# Filtro de livros disponíveis (conforme a correção anterior)
run_jsonpath_query('$.livraria.livros[?(@.disponivel == true)].titulo')

# Filtro complexo: livros com preço > 25 E da Editora Inovação
run_jsonpath_query(
    '$.livraria.livros[?(@.preco > 25 && @.editora.nome == "Editora Inovação")].titulo')

# Filtrando usuários administradores e pegando seus últimos acessos
run_jsonpath_query('$.usuarios[?(@.isAdmin == true)].ultimosAcessos')

# Filtrando livros que contenham 'Programação' na lista de tags
# jsonpath-ng não tem uma função 'contains' nativa para arrays como JMESPath.
# Para este caso, você precisaria de uma expressão mais complexa ou pós-processamento.
# Exemplo aproximado que busca por um item específico, mas não é um 'contains' genérico:
run_jsonpath_query('$.livraria.livros[?("Programação" in @.tags)].titulo')
# Nota: A sintaxe '"Programação" in @.tags' funciona para testar se um elemento está em uma lista.

# Filtrando usando regex (se o nome da editora contiver "Inovação")
# A sintaxe de regex em jsonpath-ng é `@.field =~ "regex"`.
run_jsonpath_query('$.livraria.livros[?(@.editora.nome =~ "Inovação")].titulo')


--- JSONPath Query: '$.configuracoes["chave-com-traco"]' ---
  Nenhum resultado encontrado.

--- JSONPath Query: '$.livraria.livros[0:2].titulo' ---
  Resultados (2):
    Match 1: Valor = Aprendendo JSONPath, Caminho = titulo
    Match 2: Valor = Dominando JMESPath, Caminho = titulo

--- JSONPath Query: '$.livraria.livros[0,2].titulo' ---
  Erro ao executar query: Parse error at 1:19 near token , (,)

--- JSONPath Query: '$.livraria..titulo' ---
  Resultados (5):
    Match 1: Valor = Catálogo de Livros Técnicos, Caminho = titulo
    Match 2: Valor = Aprendendo JSONPath, Caminho = titulo
    Match 3: Valor = Dominando JMESPath, Caminho = titulo
    Match 4: Valor = Introdução ao Web Scraping, Caminho = titulo
    Match 5: Valor = Python para Iniciantes, Caminho = titulo

--- JSONPath Query: '$.livraria.livros[?(@.disponivel == true)].titulo' ---
  Erro ao executar query: Error on line 1, col 18: Unexpected character: ? 

--- JSONPath Query: '$.livraria.livros[?(@.preco > 25 && @.editor

### 4. Expressões Compostas (`|`)

Você pode combinar múltiplas expressões JSONPath usando o operador de união (`|`). Isso retorna os resultados de todas as expressões combinadas.


In [6]:
# Títulos de livros OU nomes de usuários
run_jsonpath_query('$.livraria.livros[*].titulo | $.usuarios[*].nome')


--- JSONPath Query: '$.livraria.livros[*].titulo | $.usuarios[*].nome' ---
  Resultados (8):
    Match 1: Valor = Ana, Caminho = nome
    Match 2: Valor = Bruno, Caminho = nome
    Match 3: Valor = Ana, Caminho = nome
    Match 4: Valor = Bruno, Caminho = nome
    Match 5: Valor = Ana, Caminho = nome
    Match 6: Valor = Bruno, Caminho = nome
    Match 7: Valor = Ana, Caminho = nome
    Match 8: Valor = Bruno, Caminho = nome


### 5. `JsonPath` Objects (Propriedades da Expressão Compilada)

Quando você compila uma expressão com `parse()`, o objeto retornado (uma instância de `JsonPath` ou suas subclasses) tem algumas propriedades e métodos:

  - **`expr.find(data)`**: O método principal de busca, já discutido.
  - **`expr.map(data_or_list_of_data)`**: Aplica a expressão a cada item de uma lista de dados.
  - **`expr.update(data, value)`**: Atualiza o valor dos elementos encontrados.
  - **`expr.values(data)`**: Retorna apenas os valores encontrados, sem os objetos `Match`.
  - **`str(expr)`**: Retorna a string da expressão original.


In [7]:
print("--- expr.find(data) ---")

# Parse the JSONPath expression
titulo_path_expr = parse('$.livraria.titulo')

# Execute the find method
matches = titulo_path_expr.find(json_data)

if matches:
    # Access the value from the Match object
    titulo_valor = matches[0].value
    print(f"Valor do título da livraria: '{titulo_valor}'")
else:
    print("Nenhum título encontrado.")

--- expr.find(data) ---
Valor do título da livraria: 'Catálogo de Livros Técnicos'


In [8]:
# Example with multiple results
livro_titulos_expr = parse('$.livraria.livros[*].titulo')
livro_titulo_matches = livro_titulos_expr.find(json_data)

print("\nTítulos de todos os livros:")
for match in livro_titulo_matches:
    print(f"- {match.value}")


Títulos de todos os livros:
- Aprendendo JSONPath
- Dominando JMESPath
- Introdução ao Web Scraping
- Python para Iniciantes


# 1. CLASSE JSONPath - Métodos e Atributos Básicos

In [9]:
# parse() - Criar parser JSONPath
jsonpath_expr = parse('$.livraria.titulo')
match = jsonpath_expr.find(json_data)
exemplo(
    "parse() - Criar parser JSONPath",
    "parse('$.livraria.titulo').find(json_data)",
    [m.value for m in match]
)


📌 parse() - Criar parser JSONPath
Código: parse('$.livraria.titulo').find(json_data)
Resultado: ['Catálogo de Livros Técnicos']


In [10]:
# find() - Encontrar todas as ocorrências
matches = parse('$.livraria.livros[*].titulo').find(json_data)
exemplo(
    "find() - Encontrar todas as ocorrências",
    "parse('$.livraria.livros[*].titulo').find(json_data)",
    [m.value for m in matches]
)


📌 find() - Encontrar todas as ocorrências
Código: parse('$.livraria.livros[*].titulo').find(json_data)
Resultado: ['Aprendendo JSONPath', 'Dominando JMESPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']


In [11]:
# update() - Atualizar valores
data_copy = json.loads(json.dumps(json_data))  # Deep copy
parse('$.livraria.autorPrincipal').update(data_copy, 'Prof. Novo')
exemplo(
    "update() - Atualizar valores",
    "parse('$.livraria.autorPrincipal').update(data_copy, 'Prof. Novo')",
    data_copy['livraria']['autorPrincipal']
)


📌 update() - Atualizar valores
Código: parse('$.livraria.autorPrincipal').update(data_copy, 'Prof. Novo')
Resultado: Prof. Novo


In [12]:
# filter() - Filtrar matches
all_precos = parse('$.livraria.livros[*].preco').find(json_data)
filtered_precos = [m for m in all_precos if m.value > 25]
exemplo(
    "filter() - Filtrar matches (preços > 25)",
    "[m for m in parse('$.livraria.livros[*].preco').find(json_data) if m.value > 25]",
    [m.value for m in filtered_precos]
)



📌 filter() - Filtrar matches (preços > 25)
Código: [m for m in parse('$.livraria.livros[*].preco').find(json_data) if m.value > 25]
Resultado: [25.5, 30.0, 35.0]


In [13]:
# find_first() - Encontrar primeira ocorrência
# Em vez de find_first() (que não existe)
def safe_find_first(expression, data):
    matches = parse(expression).find(data)
    return matches[0] if matches else None

match = safe_find_first('$.livraria.livros[*].preco',json_data)
exemplo(
    "find_first/safe_find_first - Encontrar primeira ocorrência",
    "function",
    tr_dict({
        'value': match.value,
        'path': match.path,
        'full_path': match.full_path,
        'context': match.context,
        'type.context': type(match.context),
    }) if match else None
)


📌 find_first/safe_find_first - Encontrar primeira ocorrência
Código: function
Resultado: 
  - value: 25.5
  - path: preco
  - full_path: livraria.livros.[0].preco
  - context: DatumInContext(value={'titulo': 'Aprendendo JSONPath', 'autores': ['Alice', 'Bob'], 'ano': 2020, 'preco': 25.5, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, path=Index(index=0), context=DatumInContext(value=[{'titulo': 'Aprendendo JSONPath', 'autores': ['Alice', 'Bob'], 'ano': 2020, 'preco': 25.5, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Dominando JMESPath', 'autores': ['Charlie'], 'ano': 2021, 'preco': 30.0, 'disponivel': False, 'editora': {'nome': 'Editora Global', 'cidade': 'Rio de Janeiro'}}, {'titulo': 'Introdução ao Web Scraping', 'autores': ['David', 'Eve', 'Frank'], 'ano': 2019, 'preco': 20.0, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Python para Iniciante

In [14]:
# update_first() - Atualizar primeira ocorrência
data_copy2 = json.loads(json.dumps(json_data))
def safe_update_first(expression, data, new_value):
    """
    Simula update_first() que não existe na jsonpath-ng padrão
    Atualiza apenas o primeiro match encontrado
    Retorna True se atualizou, False se não encontrou matches
    """
    matches = parse(expression).find(data)
    if matches:
        # Pega o primeiro match
        first_match = matches[0]
        # Atualiza usando o caminho completo do primeiro match
        first_match.full_path.update(data, new_value)
        return True
    return False

safe_update_first('$.livraria.livros[*].preco',data_copy2, 99.99)
exemplo(
    "update_first() - Atualizar primeira ocorrência",
    "function",
    data_copy2['livraria']['livros'][0]['preco']
)


📌 update_first() - Atualizar primeira ocorrência
Código: function
Resultado: 99.99


# 3. OPERADORES E SINTAXES ESPECIAIS


In [15]:
# 3.1 Root ($) - Raiz do documento
root_matches = parse('$').find(json_data)
exemplo(
    "Root ($) - Raiz do documento",
    "parse('$').find(json_data)",
    f"Tipo: {type(root_matches[0].value)}, Chaves: {list(root_matches[0].value.keys())}"
)



📌 Root ($) - Raiz do documento
Código: parse('$').find(json_data)
Resultado: Tipo: <class 'dict'>, Chaves: ['livraria', 'usuarios', 'configuracoes']


In [16]:
# 3.2 Current (@) - Contexto atual (usado em filtros)
# Nota: @ é usado em expressões de filtro
current_example = ext_parse('$.livraria.livros[?(@.preco > 25)]')
exemplo(
    "Current (@) - Em filtros (extensão)",
    "Usado em expressões como: $.livraria.livros[?(@.preco > 25)]",
    "Expressão criada (precisa de ext_parse para funcionar)"
)


📌 Current (@) - Em filtros (extensão)
Código: Usado em expressões como: $.livraria.livros[?(@.preco > 25)]
Resultado: Expressão criada (precisa de ext_parse para funcionar)


In [17]:
# 3.3 Wildcard (*) - Todos os elementos
wildcard_matches = parse('$.livraria.livros[*].titulo').find(json_data)
exemplo(
    "Wildcard (*) - Todos os elementos",
    "parse('$.livraria.livros[*].titulo').find(json_data)",
    [m.value for m in wildcard_matches]
)


📌 Wildcard (*) - Todos os elementos
Código: parse('$.livraria.livros[*].titulo').find(json_data)
Resultado: ['Aprendendo JSONPath', 'Dominando JMESPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']


In [18]:
# 3.4 Recursive Descent (..) - Busca recursiva
recursive_matches = parse('$..titulo').find(json_data)
exemplo(
    "Recursive Descent (..) - Busca recursiva",
    "parse('$..titulo').find(json_data)",
    [m.value for m in recursive_matches]
)


📌 Recursive Descent (..) - Busca recursiva
Código: parse('$..titulo').find(json_data)
Resultado: ['Catálogo de Livros Técnicos', 'Aprendendo JSONPath', 'Dominando JMESPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']


In [19]:
# 3.5 Array Slice ([start:end]) - Fatias de array
slice_matches = parse('$.livraria.livros[1:3]').find(json_data)
exemplo(
    "Array Slice ([1:3]) - Fatias de array",
    "parse('$.livraria.livros[1:3]').find(json_data)",
    [m.value['titulo'] for m in slice_matches]
)


📌 Array Slice ([1:3]) - Fatias de array
Código: parse('$.livraria.livros[1:3]').find(json_data)
Resultado: ['Dominando JMESPath', 'Introdução ao Web Scraping']


In [20]:
# 3.6 Array Index ([n]) - Índice específico
index_match = parse('$.livraria.livros[2].titulo').find(json_data)
exemplo(
    "Array Index ([2]) - Índice específico",
    "parse('$.livraria.livros[2].titulo').find(json_data)",
    [m.value for m in index_match]
)


📌 Array Index ([2]) - Índice específico
Código: parse('$.livraria.livros[2].titulo').find(json_data)
Resultado: ['Introdução ao Web Scraping']


In [21]:
# 3.7 Multiple Array Indices ([1,3]) - Múltiplos índices
# multi_index = ext_parse('$.livraria.livros[0,2]').find(json_data)
multi_index = parse('$.livraria.livros[0]').find(json_data)
multi_index.extend(parse('$.livraria.livros[2]').find(json_data))
exemplo(
    "Multiple Array Indices ([0,2]) - Múltiplos índices",
    "parse('$.livraria.livros[0,2]').find(json_data)",
    [m.value['titulo'] for m in multi_index]
)


📌 Multiple Array Indices ([0,2]) - Múltiplos índices
Código: parse('$.livraria.livros[0,2]').find(json_data)
Resultado: ['Aprendendo JSONPath', 'Introdução ao Web Scraping']


In [22]:
# 3.8 Last Array Index ([-1]) - Último elemento
last_match = parse('$.livraria.livros[-1].titulo').find(json_data)
exemplo(
    "Last Array Index ([-1]) - Último elemento",
    "parse('$.livraria.livros[-1].titulo').find(json_data)",
    [m.value for m in last_match]
)


📌 Last Array Index ([-1]) - Último elemento
Código: parse('$.livraria.livros[-1].titulo').find(json_data)
Resultado: ['Python para Iniciantes']


# 4. EXTENSÕES (jsonpath_ng.ext)


In [23]:
# 4.1 Filtros com expressões
try:
    filter_expr = ext_parse('$.livraria.livros[?(@.preco > 25)]')
    filter_matches = filter_expr.find(json_data)
    exemplo(
        "Filtros com expressões [?(@.preco > 25)]",
        "ext_parse('$.livraria.livros[?(@.preco > 25)]').find(json_data)",
        [m.value['titulo'] for m in filter_matches]
    )
except:
    exemplo(
        "Filtros com expressões",
        "ext_parse('$.livraria.livros[?(@.preco > 25)]')",
        "Requer jsonpath-ng-ext para funcionar completamente"
    )



📌 Filtros com expressões [?(@.preco > 25)]
Código: ext_parse('$.livraria.livros[?(@.preco > 25)]').find(json_data)
Resultado: ['Dominando JMESPath', 'Python para Iniciantes']


In [24]:
# 4.2 Filtros com operadores de comparação
try:
    bool_filter = ext_parse('$.livraria.livros[?(@.disponivel == true)]')
    bool_matches = bool_filter.find(json_data)
    exemplo(
        "Filtros booleanos [?(@.disponivel == true)]",
        "ext_parse('$.livraria.livros[?(@.disponivel == true)]').find(json_data)",
        [m.value['titulo'] for m in bool_matches]
    )
except:
    exemplo(
        "Filtros booleanos",
        "ext_parse('$.livraria.livros[?(@.disponivel == true)]')",
        "Funcionalidade de extensão"
    )



📌 Filtros booleanos [?(@.disponivel == true)]
Código: ext_parse('$.livraria.livros[?(@.disponivel == true)]').find(json_data)
Resultado: ['Aprendendo JSONPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']


In [25]:
# 4.3 Operadores lógicos (and, or)
try:
    logic_filter = ext_parse(
        '$.livraria.livros[?(@.preco > 20 & @.disponivel == true)]')
    logic_matches = logic_filter.find(json_data)
    exemplo(
        "Operadores lógicos [?(@.preco > 20 & @.disponivel == true)]",
        "ext_parse('$.livraria.livros[?(@.preco > 20 & @.disponivel == true)]')",
        [m.value['titulo']
            for m in logic_matches] if logic_matches else "Sem matches"
    )
except:
    exemplo(
        "Operadores lógicos",
        "Exemplo: $.livraria.livros[?(@.preco > 20 & @.disponivel == true)]",
        "Funcionalidade de extensão"
    )



📌 Operadores lógicos [?(@.preco > 20 & @.disponivel == true)]
Código: ext_parse('$.livraria.livros[?(@.preco > 20 & @.disponivel == true)]')
Resultado: ['Aprendendo JSONPath', 'Python para Iniciantes']


# 5. CLASSES INTERNAS E SUBCLASSES


In [26]:
# 5.1 Fields - Acesso a campos
fields_expr = Fields('titulo')
exemplo(
    "Fields('titulo') - Classe para campos",
    "Fields('titulo')",
    f"Tipo: {type(fields_expr)}"
)



📌 Fields('titulo') - Classe para campos
Código: Fields('titulo')
Resultado: Tipo: <class 'jsonpath_ng.jsonpath.Fields'>


In [27]:
# 5.2 Index - Acesso por índice
index_expr = Index(0)
exemplo(
    "Index(0) - Classe para índices",
    "Index(0)",
    f"Tipo: {type(index_expr)}"
)



📌 Index(0) - Classe para índices
Código: Index(0)
Resultado: Tipo: <class 'jsonpath_ng.jsonpath.Index'>


In [28]:
# 5.3 Child - Relacionamento pai-filho
try:
    child_expr = Child(Fields('livraria'), Fields('titulo'))
    child_result = child_expr.find(json_data)
    exemplo(
        "Child(Fields('livraria'), Fields('titulo')) - Relacionamento pai-filho",
        "Child(Fields('livraria'), Fields('titulo')).find(json_data)",
        [m.value for m in child_result]
    )
except Exception as e:
    exemplo(
        "Child - Relacionamento pai-filho",
        "Child(Fields('livraria'), Fields('titulo'))",
        f"Erro: {str(e)}"
    )



📌 Child(Fields('livraria'), Fields('titulo')) - Relacionamento pai-filho
Código: Child(Fields('livraria'), Fields('titulo')).find(json_data)
Resultado: ['Catálogo de Livros Técnicos']


In [29]:
# 5.4 Descendant - Busca descendente
try:
    desc_expr = Descendant(Fields('livraria'), Fields('titulo'))
    desc_result = desc_expr.find(json_data)
    exemplo(
        "Descendant - Busca descendente",
        "Descendant(Fields('livraria'), Fields('titulo')).find(json_data)",
        [m.value for m in desc_result]
    )
except Exception as e:
    exemplo(
        "Descendant - Busca descendente",
        "Descendant(Fields('livraria'), Fields('titulo'))",
        f"Classe interna para busca descendente"
    )



📌 Descendant - Busca descendente
Código: Descendant(Fields('livraria'), Fields('titulo'))
Resultado: Classe interna para busca descendente


# 6. CASOS ESPECIAIS E CARACTERES ESPECIAIS


In [30]:
# 6.1 Chaves com caracteres especiais
special_key = parse("$.configuracoes['chave~com/traco']").find(json_data)
exemplo(
    "Chaves com caracteres especiais",
    "parse(\"$.configuracoes['chave~com/traco']\").find(json_data)",
    [m.value for m in special_key]
)



📌 Chaves com caracteres especiais
Código: parse("$.configuracoes['chave~com/traco']").find(json_data)
Resultado: ['valor']


In [31]:
# 6.2 Escape de caracteres
try:
    # Para chaves que podem ter pontos ou outros caracteres especiais
    escaped_example = parse('$.configuracoes.tema').find(json_data)
    exemplo(
        "Chaves normais (sem escape necessário)",
        "parse('$.configuracoes.tema').find(json_data)",
        [m.value for m in escaped_example]
    )
except Exception as e:
    exemplo(
        "Escape de caracteres",
        "parse('$.configuracoes.tema')",
        f"Erro: {str(e)}"
    )



📌 Chaves normais (sem escape necessário)
Código: parse('$.configuracoes.tema').find(json_data)
Resultado: ['escuro']


# 7. MÉTODOS UTILITÁRIOS E ANÁLISE


In [32]:
# 7.1 str() - Representação string
expr = parse('$.livraria.livros[*].titulo')
exemplo(
    "str(expression) - Representação string",
    "str(parse('$.livraria.livros[*].titulo'))",
    str(expr)
)



📌 str(expression) - Representação string
Código: str(parse('$.livraria.livros[*].titulo'))
Resultado: $.livraria.livros.[*].titulo


In [33]:
# 7.2 repr() - Representação técnica
exemplo(
    "repr(expression) - Representação técnica",
    "repr(parse('$.livraria.livros[*].titulo'))",
    repr(expr)
)


📌 repr(expression) - Representação técnica
Código: repr(parse('$.livraria.livros[*].titulo'))
Resultado: Child(Child(Child(Child(Root(), Fields('livraria')), Fields('livros')), Slice(start=None,end=None,step=None)), Fields('titulo'))


In [34]:
# 7.3 Análise de caminho complexo
complex_path = parse('$..editora.nome')
complex_matches = complex_path.find(json_data)
exemplo(
    "Análise de caminho complexo ($..editora.nome)",
    "parse('$..editora.nome').find(json_data)",
    [m.value for m in complex_matches]
)


📌 Análise de caminho complexo ($..editora.nome)
Código: parse('$..editora.nome').find(json_data)
Resultado: ['Editora Inovação', 'Editora Global', 'Editora Inovação']


# 8. COMBINAÇÕES E PADRÕES AVANÇADOS


In [35]:
# 8.1 União de caminhos (usando |)
try:
    union_expr = parse('$.livraria.titulo | $.livraria.autorPrincipal')
    union_matches = union_expr.find(json_data)
    exemplo(
        "União de caminhos (titulo | autorPrincipal)",
        "parse('$.livraria.titulo | $.livraria.autorPrincipal').find(json_data)",
        [m.value for m in union_matches]
    )
except:
    # Alternativa manual
    titulo_matches = parse('$.livraria.titulo').find(json_data)
    autor_matches = parse('$.livraria.autorPrincipal').find(json_data)
    exemplo(
        "União manual de caminhos",
        "Combinação de duas consultas separadas",
        [m.value for m in titulo_matches + autor_matches]
    )



📌 União de caminhos (titulo | autorPrincipal)
Código: parse('$.livraria.titulo | $.livraria.autorPrincipal').find(json_data)
Resultado: ['Dr. Tech']


In [36]:
# 8.2 Aninhamento profundo
deep_matches = parse('$.livraria.livros[*].editora.cidade').find(json_data)
exemplo(
    "Aninhamento profundo (livros -> editora -> cidade)",
    "parse('$.livraria.livros[*].editora.cidade').find(json_data)",
    [m.value for m in deep_matches]
)


📌 Aninhamento profundo (livros -> editora -> cidade)
Código: parse('$.livraria.livros[*].editora.cidade').find(json_data)
Resultado: ['São Paulo', 'Rio de Janeiro', 'São Paulo']


In [37]:
# 8.3 Arrays dentro de objetos
autores_matches = parse('$.livraria.livros[*].autores[*]').find(json_data)
exemplo(
    "Arrays dentro de objetos (todos os autores)",
    "parse('$.livraria.livros[*].autores[*]').find(json_data)",
    [m.value for m in autores_matches]
)


📌 Arrays dentro de objetos (todos os autores)
Código: parse('$.livraria.livros[*].autores[*]').find(json_data)
Resultado: ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace']


# 9. TRATAMENTO DE ERROS E CASOS EXTREMOS


In [38]:
# 9.1 Caminho inexistente
inexistente = parse('$.caminho.inexistente').find(json_data)
exemplo(
    "Caminho inexistente",
    "parse('$.caminho.inexistente').find(json_data)",
    f"Resultado: {len(inexistente)} matches encontrados"
)


📌 Caminho inexistente
Código: parse('$.caminho.inexistente').find(json_data)
Resultado: Resultado: 0 matches encontrados


In [39]:
# 9.2 Índice fora do range
try:
    fora_range = parse('$.livraria.livros[10]').find(json_data)
    exemplo(
        "Índice fora do range [10]",
        "parse('$.livraria.livros[10]').find(json_data)",
        f"Resultado: {len(fora_range)} matches encontrados"
    )
except Exception as e:
    exemplo(
        "Índice fora do range",
        "parse('$.livraria.livros[10]')",
        f"Tratamento silencioso: retorna lista vazia"
    )


📌 Índice fora do range [10]
Código: parse('$.livraria.livros[10]').find(json_data)
Resultado: Resultado: 0 matches encontrados


# PERFORMANCE E OTIMIZAÇÃO


In [40]:
# Reutilização de expressões
expr_reutilizavel = parse('$.livraria.livros[*].preco')
resultado1 = expr_reutilizavel.find(json_data)
resultado2 = expr_reutilizavel.find(json_data)  # Reutilização
exemplo(
    "Reutilização de expressões",
    "expr = parse('$.livraria.livros[*].preco'); expr.find(data)",
    f"Mesma expressão usada {2} vezes"
)


📌 Reutilização de expressões
Código: expr = parse('$.livraria.livros[*].preco'); expr.find(data)
Resultado: Mesma expressão usada 2 vezes


In [41]:
# find_first() vs find() para um resultado
start = time.time()
primeiro = safe_find_first('$.livraria.livros[*].titulo',json_data)
tempo_first = time.time() - start

start = time.time()
todos = parse('$.livraria.livros[*].titulo').find(json_data)
tempo_all = time.time() - start

exemplo(
    "find_first() vs find() - Performance",
    "Comparação de tempos de execução",
    f"find_first(): {tempo_first:.6f}s, find(): {tempo_all:.6f}s"
)


📌 find_first() vs find() - Performance
Código: Comparação de tempos de execução
Resultado: find_first(): 0.018604s, find(): 0.018632s


# RESUMO FINAL

- PRINCIPAIS MÉTODOS DA CLASSE JSONPath:
   - `parse(expression)`         - Criar parser JSONPath
   - `find(data)`                - Encontrar todas as ocorrências
   - `find_first(data)`          - Encontrar primeira ocorrência
   - `update(data, value)`       - Atualizar todos os valores
   - `update_first(data, value)` - Atualizar primeiro valor

- ATRIBUTOS DE DatumInContext (resultado das buscas):
   - `.value`      - Valor encontrado
   - `.path`       - Caminho JSONPath
   - `.full_path`  - Caminho completo
   - `.context`    - Contexto do match

- OPERADORES JSONPATH:
   - `$`           - Raiz do documento
   - `@`           - Contexto atual (em filtros)
   - `*`           - Wildcard (todos os elementos)
   - `..`          - Busca recursiva (descendant)
   - `[n]`         - Índice específico
   - `[start:end]` - Slice de array
   - `[a,b,c]`     - Múltiplos índices
   - `['key']`     - Chave com caracteres especiais

- EXTENSÕES (jsonpath_ng.ext):
   - `[?(@.prop > value)]`    - Filtros com comparação
   - `[?(@.prop == value)]`   - Filtros de igualdade
   - `[?(@.a > 1 & @.b < 5)]` - Operadores lógicos

- CLASSES INTERNAS:
   - `Fields()`     - Acesso a campos
   - `Index()`      - Acesso por índice
   - `Child()`      - Relacionamento pai-filho
   - `Descendant()` - Busca descendente
   - `Union()`      - União de expressões

- DICAS DE PERFORMANCE:
   - Reutilize expressões compiladas
   - Use find_first() quando precisar apenas do primeiro resultado
   - Prefira caminhos específicos a buscas recursivas quando possível

## Exceções Comuns

  - **`jsonpath_ng.exceptions.JsonPathLexerError`**: Ocorre quando a expressão JSONPath contém caracteres inesperados ou está malformada sintaticamente (erros de tokenização).
  - **`jsonpath_ng.exceptions.JsonPathParserError`**: Ocorre quando a expressão JSONPath é sintaticamente válida, mas semanticamente incorreta ou segue uma estrutura que o parser não entende.
